In [1]:
import random
import pandas as pd

# Step 1: Random chromosome length
chromosome_length = random.randint(1000, 2000)

# Step 2: Define genes and assign random positions
genes = [f"G{i+1}" for i in range(20)]
positions = sorted(random.sample(range(1, chromosome_length), 20))
gene_positions = list(zip(genes, positions))

# Step 3: Create two homologs with different versions (A and B)
def create_chromosome(suffix):
    return [(f"{gene}{suffix}", pos) for gene, pos in gene_positions]

chr1 = create_chromosome("_A")
chr2 = create_chromosome("_B")

# Step 4: Choose a random crossover point
crossover_point = random.randint(1, chromosome_length)
swap_above = random.random() > 0.5  # True = swap genes ABOVE point, False = swap genes BELOW

# Step 5: Recombine based on crossover rule
recombinant1 = []
recombinant2 = []

for (g1, pos), (g2, _) in zip(chr1, chr2):
    if (swap_above and pos > crossover_point) or (not swap_above and pos < crossover_point):
        recombinant1.append((g2, pos))
        recombinant2.append((g1, pos))
    else:
        recombinant1.append((g1, pos))
        recombinant2.append((g2, pos))

# Step 6: Output as DataFrame
df = pd.DataFrame({
    "Gene": genes,
    "Position": positions,
    "Chr1_Before": [g for g, _ in chr1],
    "Chr2_Before": [g for g, _ in chr2],
    "Chr1_After": [g for g, _ in recombinant1],
    "Chr2_After": [g for g, _ in recombinant2],
})

print(f"Crossover point: {crossover_point} (swap {'above' if swap_above else 'below'})\n")
print(df)


Crossover point: 54 (swap below)

   Gene  Position Chr1_Before Chr2_Before Chr1_After Chr2_After
0    G1        29        G1_A        G1_B       G1_B       G1_A
1    G2       191        G2_A        G2_B       G2_A       G2_B
2    G3       214        G3_A        G3_B       G3_A       G3_B
3    G4       221        G4_A        G4_B       G4_A       G4_B
4    G5       247        G5_A        G5_B       G5_A       G5_B
5    G6       521        G6_A        G6_B       G6_A       G6_B
6    G7       573        G7_A        G7_B       G7_A       G7_B
7    G8       590        G8_A        G8_B       G8_A       G8_B
8    G9       804        G9_A        G9_B       G9_A       G9_B
9   G10       830       G10_A       G10_B      G10_A      G10_B
10  G11      1075       G11_A       G11_B      G11_A      G11_B
11  G12      1133       G12_A       G12_B      G12_A      G12_B
12  G13      1134       G13_A       G13_B      G13_A      G13_B
13  G14      1147       G14_A       G14_B      G14_A      G14_B
14  G1